In [1]:
## installing all the required libraries

!pip install llama-index
!pip install pypdf
!pip install openai

In [4]:
# command to cross check if llama_index is installed or not
!pip show llama_index

Name: llama-index
Version: 0.10.56
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: c:\users\91983\anaconda3\lib\site-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-legacy, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse
Required-by: 


In [5]:
## importing neccessary library
import llama_index
import openai
from llama_index.core import SimpleDirectoryReader ,VectorStoreIndex
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.indices.postprocessor import SimilarityPostprocessor
from llama_index.core.query_engine import RetrieverQueryEngine
from IPython.display import display, HTML



In [6]:
'''
This is the initial setup method
This method is used to create a documents and then get document parsed to node ,which is then 
indexed using vectorstoreindex class and then we create a query engine out of index. 

'''

def init():
    global queryEngine
    # loading data through reader and create a documnet
    reader = SimpleDirectoryReader(input_dir="./dataPDF")
    documents = reader.load_data()

    # create parser and parse document into nodes
    parser = SimpleNodeParser.from_defaults()
    nodes = parser.get_nodes_from_documents(documents=documents)

    #  build index
    index  = VectorStoreIndex(nodes)
    # Construct Query Engine
    queryEngine = index.as_query_engine()
   



In [7]:



## Query response function
'''
This method takes user input and get result from query engine and analyze it and then it produce
result in a given format that includes <response fetched>+<document from where this reponse is fetched>  
'''
def query_response(user_input):
    response = queryEngine.query(user_input)
    analyzed_response = get_response(response.response)
    final_response = 'I am not having sufficent resource available.Please connect with admin for any information on it.'
    if analyzed_response == 'Positive':
        file_name = response.source_nodes[0].node.metadata['file_name']
        final_response = response.response + '\n Check further at ' + file_name + ' document'

    return final_response

In [8]:
'''
This method is used  to classify query engine response  sentiment as Positive or Negative
'''
def get_response(example):
    
    response = openai.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
      {
        "role": "system",
        "content": "You will be provided with a paragraph, and your task is to classify its sentiment as Positive or Negative"
      },
      {
        "role": "user",
        "content": "I'm sorry, but I am unable to provide a relevant answer to the query gvchj based on the context information provided"
      },
      {
        "role": "assistant",
        "content": "Negative"
      },
      {
        "role": "user",
        "content": "I'm sorry, I couldn't find any relevant information or answer to the query provided."
      },
      {
        "role": "assistant",
        "content": "Negative"
      },
      {
        "role": "user",
        "content": "To create a permit application, you need to define permit types by setting up permit categories, defining permit statuses, setting up workflows and fee models, creating permit types, designing permit application forms, and more"
      },
      {
        "role": "assistant",
        "content": "Positive"
      },
      {
        "role": "user",
        "content": "To delete a contact, you should follow these steps:\n1. Select Common Setup > Contact Type.\n2. Click a row on the Contact Type page."
      },
      {
        "role": "assistant",
        "content": "Positive"
      },
      {
        "role": "user",
        "content": "Oracle Corporation and its affiliates are not responsible for and expressly disclaim all warranties of any kind with respect to third-party content, products, and services unless otherwise set forth in an applicable agreement between you and Oracle"
      },
      {
        "role": "assistant",
        "content": "Positive"
      },
      {
        "role": "user",
        "content": "I'm sorry, I couldn't find any relevant information or answer to the query provided. If you have any other questions or need assistance, feel free to ask."
      },
      {
        "role": "assistant",
        "content": "Negative"
      },
     {
      "role": "user",
      "content": f"""{example} """
    }
    ]
   )
    return response.choices[0].message.content


In [11]:
'''
This is the conversation ini
'''

def initialize_conv():
    # calling the init method so that our query engine is ready
    init()
    print("Hi! I am Ivan ,How can I help you on Oracle doc.?write exit once you are done.")
    
    while True:
        input_qry = input("Enter your query here    ")
        if input_qry.lower() == 'exit':
            print('Thanks for your Time.Bye')
            break
        else:
            response = query_response(input_qry)
            display(HTML(f'<p style="font-size:20px">{response}</p>'))

In [12]:
initialize_conv()

Hi! I am Ivan ,How can I help you on Oracle doc.?write exit once you are done.
Enter your query here    How to register a new user from public user access page


Enter your query here    how to smoke cigarette


Enter your query here    how to create a contact


Enter your query here    how is weather today 


Enter your query here    drinking water


Enter your query here    exit
Thanks for your Time.Bye
